In [4]:
import torch.nn as nn

In [1]:
import torch

In [3]:
# 实现lstm和lstmp的源码
# 定义常量
bs, T, i_size, h_size = 2, 3, 4, 5
input = torch.randn(bs,T, i_size)
c0 = torch.randn(bs, h_size)
h0 = torch.randn(bs, h_size)

In [5]:
lstm_layer = nn.LSTM(i_size, h_size, batch_first=True)
outout, (h_n, c_n) = lstm_layer(input,(h0.unsqueeze(0), c0.unsqueeze(0)))

In [6]:
outout

tensor([[[ 0.1260, -0.0432,  0.0379,  0.3549,  0.1202],
         [ 0.0614,  0.1587,  0.0958,  0.1778, -0.1328],
         [-0.0757,  0.0426,  0.1516,  0.0110, -0.1496]],

        [[ 0.2533, -0.1454,  0.2498,  0.2968,  0.1513],
         [ 0.0933, -0.0966,  0.2524,  0.0680, -0.0801],
         [ 0.1399, -0.1651,  0.1841,  0.2158, -0.1731]]],
       grad_fn=<TransposeBackward0>)

In [8]:
for k, v in lstm_layer.named_parameters():
    print(k,v.shape)

weight_ih_l0 torch.Size([20, 4])
weight_hh_l0 torch.Size([20, 5])
bias_ih_l0 torch.Size([20])
bias_hh_l0 torch.Size([20])


In [9]:
# 自己写一个LSTM模型
def lstm_forward(input, initial_states, w_ih, w_hh, b_ih, b_hh):
    h0, c0 = initial_states
    bs, T, i_size = input.shape
    prev_h = h0
    prec_c = c0
    w_ih #(4*h_size, i_size)
    w_hh #(4*h_size, h_size)

    batch_w_ih = w_ih.unsqueeze(0).tile(bs,1,1) #
    batch_w_hh = w_hh.unsqueeze(0).tile(bs,1,1)
    output_size = h_size
    outout = torch.zeros(bs, T, output_size) #输出序列

    for t in range(T):
        x = input[:,t,:] #当前时刻的输入向量[bs, i_size]
        w_times_x = torch.bmm(batch_w_ih, x.unsqueeze(-1)) #[bs,4*h_size,1]
        w_times_x = w_times_x.squeeze(-1)
        w_times_h_prev = torch.bmm(batch_w_hh, prev_h.unsqueeze(-1))
        w_times_h_prev = w_times_h_prev.squeeze(-1)

        # 分别计算输入们(i)、遗忘门(f)，cell门(g)、输出门(0)
        i_t = torch.sigmoid(w_times_x[:,:h_size] + w_times_h_prev[:,:h_size] + b_ih[:h_size] + b_hh[:h_size])
        f_t = torch.sigmoid(w_times_x[:,h_size:2*h_size] + w_times_h_prev[:,h_size:2*h_size] + b_ih[h_size:2*h_size] + b_hh[h_size:2*h_size])
        g_t = torch.tanh(w_times_x[:,2*h_size:3*h_size] + w_times_h_prev[:,2*h_size:3*h_size] + b_ih[2*h_size:3*h_size] + b_hh[2*h_size:3*h_size])
        o_t = torch.sigmoid(w_times_x[:,3*h_size:4*h_size] + w_times_h_prev[:,3*h_size:4*h_size] + b_ih[3*h_size:4*h_size] + b_hh[3*h_size:4*h_size])

        prev_c = f_t*prec_c + i_t*g_t
        prev_h = o_t*torch.tanh(prev_c)

        outout[:,t,:] = prev_h

    return outout, (prev_h, prev_c)

output_custom, (h_final_custom, c_final_custom) = lstm_forward(input, (h0,c0), lstm_layer.weight_ih_l0, lstm_layer.weight_hh_l0,lstm_layer.bias_ih_l0, lstm_layer.bias_hh_l0)

In [11]:
print(output_custom)

tensor([[[ 0.1260, -0.0432,  0.0379,  0.3549,  0.1202],
         [ 0.0650,  0.2983,  0.0375,  0.1358,  0.1681],
         [-0.0190,  0.0811,  0.1171,  0.0596,  0.1556]],

        [[ 0.2533, -0.1454,  0.2498,  0.2968,  0.1513],
         [ 0.1153, -0.1831,  0.2243,  0.0433, -0.1640],
         [ 0.1997, -0.2824,  0.1481,  0.3214, -0.0528]]], grad_fn=<CopySlices>)


In [ ]:
for k,v in output_custom.